# Shared code block

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Lambda
from tensorflow.keras.optimizers import Adam

IMAGE_SIZE = (256, 256)

def load_images_from_folder(folder, size=IMAGE_SIZE):
    imgs = []
    files = sorted([
        f for f in os.listdir(folder)
        if f.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff"))
    ])

    for fname in files:
        path = os.path.join(folder, fname)
        img = cv2.imread(path, cv2.IMREAD_COLOR)
        if img is None:
            print("Skipping unreadable:", fname)
            continue

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, size)
        img = img.astype("float32") / 255.0
        imgs.append(img)

    return np.array(imgs)

# ---- Custom clip layer replacing Lambda ----
class Clip01(tf.keras.layers.Layer):
    def call(self, x):
        return tf.clip_by_value(x, 0.0, 1.0)


def build_autoencoder(input_shape=(256, 256, 3)):
    inp = Input(shape=input_shape)

    # ----- Encoder -----
    x = Conv2D(32, (3,3), activation="relu", padding="same")(inp)
    x = MaxPooling2D((2,2), padding="same")(x)

    x = Conv2D(64, (3,3), activation="relu", padding="same")(x)
    x = MaxPooling2D((2,2), padding="same")(x)

    # ----- Bottleneck -----
    x = Conv2D(128, (3,3), activation="relu", padding="same")(x)

    # ----- Decoder -----
    x = UpSampling2D((2,2))(x)
    x = Conv2D(64, (3,3), activation="relu", padding="same")(x)

    x = UpSampling2D((2,2))(x)
    x = Conv2D(32, (3,3), activation="relu", padding="same")(x)

    # Raw output before skip
    out_raw = Conv2D(3, (3,3), activation="sigmoid", padding="same")(x)

    # Skip connection
    merged = out_raw + inp

    # Clipping using custom layer (NO lambda!)
    out = Clip01()(merged)

    model = Model(inp, out)
    model.compile(optimizer=Adam(1e-3), loss="mae")

    return model

In [ ]:
# blur
BASE_INPUT = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/input"
STYLE_FOLDER = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/style/blur"
MODEL_SAVE_DIR = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/models"

os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

print("Loading input images...")
input_images = load_images_from_folder(BASE_INPUT)
print(input_images.shape)

print("Loading BLUR style images...")
blur_images = load_images_from_folder(STYLE_FOLDER)
print(blur_images.shape)

if len(input_images) != len(blur_images):
    raise ValueError("Mismatch in BLUR training set!")

model = build_autoencoder()
model.fit(
    input_images,
    blur_images,
    epochs=200,
    batch_size=4,
    validation_split=0.2,
    verbose=1
)

model.save(f"{MODEL_SAVE_DIR}/autoencoder_blur.keras")
print("Saved BLUR model.")


In [ ]:
# night_vis

import os

BASE_INPUT = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/input"
STYLE_FOLDER = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/style/night_vis"
MODEL_SAVE_DIR = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/models"

os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

print("Loading input images...")
input_images = load_images_from_folder(BASE_INPUT)

print("Loading Night Vision style images...")
night_vis_images = load_images_from_folder(STYLE_FOLDER)

if len(input_images) != len(night_vis_images):
    raise ValueError("Mismatch in Night Vision training set!")

model = build_autoencoder()
model.fit(
    input_images,
    night_vis_images,
    epochs=50,
    batch_size=4,
    validation_split=0.2,
    verbose=1
)

model.save(f"{MODEL_SAVE_DIR}/autoencoder_night_vis.keras")
print("Saved  Night Vision model.")


In [4]:

# outline
import os

BASE_INPUT = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/input"
STYLE_FOLDER = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/style/outline"
MODEL_SAVE_DIR = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/models"

os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

print("Loading input images...")
input_images = load_images_from_folder(BASE_INPUT)

print("Loading OUTLINE style images...")
outline_images = load_images_from_folder(STYLE_FOLDER)

if len(input_images) != len(outline_images):
    raise ValueError("Mismatch in OUTLINE training set!")

model = build_autoencoder()
model.fit(
    input_images,
    outline_images,
    epochs=50,
    batch_size=4,
    validation_split=0.2,
    verbose=1
)

model.save(f"{MODEL_SAVE_DIR}/autoencoder_outline.keras")
print("Saved OUTLINE model.")


KeyboardInterrupt: 

In [ ]:
# poster style
import os

BASE_INPUT = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/input"
STYLE_FOLDER = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/style/poster"
MODEL_SAVE_DIR = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/models"

os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

print("Loading input images...")
input_images = load_images_from_folder(BASE_INPUT)

print("Loading POSTER style images...")
poster_images = load_images_from_folder(STYLE_FOLDER)

if len(input_images) != len(poster_images):
    raise ValueError("Mismatch in POSTER training set!")

model = build_autoencoder()
model.fit(
    input_images,
    poster_images,
    epochs=50,
    batch_size=4,
    validation_split=0.2,
    verbose=1
)

model.save(f"{MODEL_SAVE_DIR}/autoencoder_poster.keras")
print("Saved POSTER model.")
